#  RAG and Python function Deployment

In this notebook, we create and deploy a python function that takes a user defined question as input and generates an answer using the RAG process. The function does the following:
- Connects to Elasticsearch and retrieves the top N results (over a certain threshold) matching the question from our vector index
- Includes the question, vector search results into an optimized prompt template for the LLM
- Runs LLM inference on watsonx.ai to generate an answer
- Checks for hallucinations using a word overlap algorithm and threshold values


## Contents

This notebook contains the following parts:

- [Pre-Requisite Libraries and Dependencies](#Setup)
- [Import Parameter Set](#parameterimport)
- [Initialise Deployment Space](#DeploymentSpace)
- [Promote Assets to Deployment Space](#promote)
- [Create Scoring Function](#ScoringFunction)
- [Deploy Scoring Function in Deployment Space](#DeployScoringFunction)
- [Test the scoring function](#scoring)
- [Update the feedback log](#feedback-logging)
- [Update parameter set in the project & deployment space](#updateParameters)




<a id="Setup"></a>
### Pre-Requisite Libraries and Dependencies

Download and import mandatory libraries and dependencies

In [ ]:
!pip install --upgrade elasticsearch | tail -n 1
!pip install ibm_watsonx_ai==1.0.4  | tail -n 1

In [ ]:
import os
import json
import string
import time
import uuid
from ibm_watsonx_ai import APIClient
import warnings
warnings.filterwarnings("ignore")

from ibm_watsonx_ai.foundation_models.prompts import PromptTemplate, PromptTemplateManager
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, PromptTemplateFormats

In [ ]:
project_id=os.environ['PROJECT_ID']
# Environment and host url
hostname = os.environ['RUNTIME_ENV_APSX_URL']

if hostname.endswith("cloud.ibm.com") == True:
    environment = "cloud"
else:
    environment = "on-prem"
    from ibm_watson_studio_lib import access_project_or_space
    wslib = access_project_or_space()




<a id="parameterimport"></a>
### Import Parameter Sets



In [ ]:
parameter_sets = ["RAG_parameter_set","RAG_advanced_parameter_set"]
parameters={}
for parameter_set in parameter_sets:
    parameter_vals= wslib.assets.get_asset(parameter_set, "parameter_set", raw=True)
    if 'entity' in parameter_vals and 'parameter_set' in parameter_vals['entity'] and 'parameters' in parameter_vals['entity']['parameter_set']:
        parameter_vals = parameter_vals['entity']
    params = {param['name']: param['value'] for param in parameter_vals['parameter_set']['parameters']}
    parameters.update(params)    

Below cell sets the `project id, hostname and watsonx.ai environment`

In [ ]:
ibm_api_key=parameters['watsonx_ai_api_key']
if environment == "cloud":
    WML_SERVICE_URL = f"https://{hostname.split('.')[1]}.ml.cloud.ibm.com" if hostname.split('.')[0].endswith("api") and hostname.split('.')[2] == "dataplatform" else "https://us-south.ml.cloud.ibm.com"
    wml_credentials = {"apikey": ibm_api_key, "url": WML_SERVICE_URL}
    token=""
else:
    token = os.environ['USER_ACCESS_TOKEN']
    wml_credentials = {"token": token,"instance_id" : "openshift","url": hostname}

In [ ]:
try:
    filename = 'rag_helper_functions.py'
    wslib.download_file(filename)
    import rag_helper_functions
    print("rag_helper_functions imported from the project assets")
except NameError as e:
    print(str(e))
    print("If running watsonx.ai aaS on IBM Cloud, check that the first cell in the notebook contains a project token. If not, select the vertical ellipsis button from the notebook toolbar and `insert project token`. Also check that you have specified your ibm_api_key in the second code cell of the notebook")


### Set Watsonx.ai client
Below cell uses the watson machine learning credentials to create an API client to interact with the project and deployment space. 

In [ ]:
client = APIClient(wml_credentials)
client.set.default_project(project_id=project_id)

### Import Prompt Template

Imports the prompt template based on the parameter set by the user. 

In [ ]:

prompt_mgr = PromptTemplateManager(
                credentials=wml_credentials,
                project_id=project_id
                )


df_prompt = prompt_mgr.list()
prompt_template_id=df_prompt.loc[df_prompt['NAME'] == parameters['llm_prompt_template_file'], 'ID'].values[0]
print("Currently using the ", parameters['llm_prompt_template_file'], "prompt template.")

 
<a id="DeploymentSpace"></a>
### Get the Deployment Space details and set default space.

In the next steps we save and deploy the pipeline. The pipeline can be saved and deployed in the same way we save and deploy models.

Before we deploy a function we must create a deployment space. Watson Machine Learning provides deployment spaces where the user can save, configure and deploy their models. We can save models, functions and data assets in this space.

**Creating a Deployment Space** <br>
Before we save the function we must create a deployment space. Watson Machine Learning provides deployment spaces where the user can save, configure and deploy their functions or models. We can save models, functions and data assets in this space.
If you do not have space already created, you can use Deployment Spaces Dashboard to create one. Follow the steps : 
* Navigate to Deployments
* Click New Deployment Space
* Enter Deployment space name, for eg : **'RAG ES Deployment Space'**

**In case of watsonx as a service**
* Follow the steps to create the space same as above. 
* Select Cloud Object Storage
* Select Watson Machine Learning instance and press Create
* `Under Manage project` > `Services & Integrations` ensure that your WML service that you provisioned is associated there. 


The steps involved in saving and deploying the pipeline are detailed in the following cells. We will use the `ibm-watson-machine-learning` package to complete these steps.

**Setting space id in parameters set**
* In your Deployment Space, copy the space ID by going to **Manage >** copy the Space GUID
* In your Project, open the `RAG_parameter_set` found in **Data Asset > Configuration**.
* Edit the parameter name `watsonx_ai_space_id` and paste the guid here.

In [ ]:
space_uid = parameters['watsonx_ai_space_id']
    
space = client.set.default_space(space_uid)
if space != 'SUCCESS':
    print("Please ensure that the space exists & try again.")
else:
    print("Default WML Client setting : ", space)

<a id="promote"></a>
### Promote Necessary Assets to the Deployment Space.

The following assets are promoted to the deployment space to be used in the deployed function
* Elasticsearch Connection
* Parameter sets
* LLM Prompt template 
* RAG Helper functions python script

In [ ]:
connection_name=parameters["elastic_search_connection_asset"]
if(next((conn for conn in wslib.list_connections() if conn['name'] == connection_name), None)):
    print(connection_name, "Connection found in the project")
    es_connection=wslib.get_connection(connection_name)
    print("Successfully retrieved the connection details")
    connection_id = client.spaces.promote(es_connection['.']['asset_id'], project_id, space_uid)
    print("Promoted the connection to deployment space")
else:
    es_connection=""
    connection_id = ""
    raise ValueError(f"No connection named {connection_name} found in the project.")

In [ ]:
log_connection_name=parameters["log_elastic_search_connection_asset"]
if(next((conn for conn in wslib.list_connections() if conn['name'] == log_connection_name), None)):
    print(log_connection_name, "Log connection found in the project")
    log_es_connection = wslib.get_connection(log_connection_name)
    print("Successfully retrieved the log connection details")
    log_connection_id = client.spaces.promote(log_es_connection['.']['asset_id'], project_id, space_uid)
    print("Promoted the connection to deployment space")
    
else:
    log_es_connection=""
    log_connection_id = ""
    print(f"No log connection named {log_connection_name} found in the project. Remove log connection name from parameter set to disable logging." \
          if not log_connection_name =='' else f"No log connection specified. Logging will be disabled.")
    

Promote the assets and the prompt template to the space to be used by the deployed function.

In [ ]:
promote_files_to_space=['rag_helper_functions.py']
space_asset_dict={}
for file in wslib.assets.list_assets('data_asset'):
    if file['name'] in promote_files_to_space:
        space_asset_dict[file['name']] = client.spaces.promote(file['asset_id'], project_id, space_uid)
        print(file['name'],"promoted to deployment space")

for parms in wslib.assets.list_assets('parameter_set'):
    if parms['name'] in parameter_sets:
        space_asset_dict[parms['name']] = client.spaces.promote(parms['asset_id'], project_id, space_uid)
        print(parms['name'],"promoted to deployment space")

space_asset_dict       

In [ ]:
print("promoting prompt to deployment space")
client.spaces.promote(prompt_template_id, project_id, space_uid)

Asset details of all data assets promoted to the space to be used by the deployed function

In [ ]:
# common ai parameters passed to deployed function
ai_params = {'space_id': space_uid, 'space_asset_dict': space_asset_dict, 'hostname': hostname, 'environment': environment, 'project_id': project_id,
             'ibm_api_key': ibm_api_key, 'token': token, 'connection_name': connection_name, 'connection_id': connection_id,
             'log_connection_name': log_connection_name, 'log_connection_id': log_connection_id,
             'log_pii_removal': ('log_pii_removal' in parameters and parameters['log_pii_removal'].lower() == 'true') }

<a id="ScoringFunction"></a>

### Python scoring pipeline function

The function handles the following tasks:

1. Initializing the WML client.
2. Downloading assets promoted to the deployment space.
3. Calling the Python helper functions script.
4. Initializing the Elasticsearch client and creating a vector store for Elasticsearch.
5. Initializing LLM with parameters.
6. Creating an LLM chain using LLM, vector store, and prompt template.
7. Retrieving a response from the LLM using LLM Chain and checking for hallucinations.
8. Writing response data to a log record in Elasticsearch and creating an index for logs if it doesn't exist.
9. Returning the result to the user.
10. Providing the document ID of the log record.
11. Updating the log record with the user's feedback (through a separate function call).

In [ ]:
def rag_scoring_pipeline_function(params=ai_params):
    import subprocess
    subprocess.check_output("pip install --upgrade elasticsearch --user", stderr=subprocess.STDOUT, shell=True)
    subprocess.check_output("pip install --upgrade sentence_transformers --user", stderr=subprocess.STDOUT, shell=True)
    subprocess.check_output("pip install langchain==0.0.354 --user", stderr=subprocess.STDOUT, shell=True)
    subprocess.check_output("pip install --upgrade langchain-community --user", stderr=subprocess.STDOUT, shell=True) 
    subprocess.check_output("pip install --upgrade langchain-core --user", stderr=subprocess.STDOUT, shell=True)
    subprocess.check_output("pip install pydantic==1.10.11 --user", stderr=subprocess.STDOUT, shell=True)
    subprocess.check_output("pip install langchain-elasticsearch --user", stderr=subprocess.STDOUT, shell=True)
    subprocess.check_output("pip install ibm-watsonx-ai==1.0.4 --user", stderr=subprocess.STDOUT, shell=True)
    subprocess.check_output("pip install langchain_ibm==0.1.7 --user", stderr=subprocess.STDOUT, shell=True)
    
    import requests
    import os
    import json    
    import warnings
    import time
    from datetime import datetime
    import re    
    import elasticsearch 
    from elasticsearch import Elasticsearch
    import pandas as pd
    from ibm_watsonx_ai import APIClient
    from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
    from ibm_watsonx_ai.foundation_models import Model
    from ibm_watsonx_ai.foundation_models.prompts import PromptTemplate, PromptTemplateManager
    from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, PromptTemplateFormats, DecodingMethods
    from ibm_watsonx_ai import Credentials
    from ibm_watson_studio_lib import access_project_or_space
    from sentence_transformers import SentenceTransformer, util
    import string
    from langchain.chains import LLMChain
    from ibm_watsonx_ai.foundation_models.extensions.rag import VectorStore
    from langchain_core.output_parsers import StrOutputParser
    from langchain_core.runnables import RunnablePassthrough
    from langchain_core.runnables import RunnableParallel
    
    s = requests.Session()

    # Conditional import, since watson_nlp might not be available in runtime environment
    watson_nlp_available = False
    if params['log_pii_removal']:
        try:
            import watson_nlp
            watson_nlp_available = True
        except ImportError:
            pass
        
    #from langchain.globals import set_debug
    #set_debug(True)
    
    
    # AI Params passsed to function
    
    hostname = params['hostname'] # url
    project_id = params['project_id'] # RAG project id
    ibm_api_key = params['ibm_api_key'] #ibm cloud api_key
    environment = params['environment'] # cloud/on-premise
    token = params['token'] # wml token
    space_asset_dict = params['space_asset_dict']
    
    
    
    # Function to initialize wml client
    def get_WML_client(environment, token, hostname):
        if environment == "on-prem":
            wml_credentials = {
               "token": token,
               "instance_id" : "openshift",
               "url": hostname
            }
        elif environment == "cloud":
            l = hostname.split(".")
            if l[0].endswith("api") and l[2]=="dataplatform":
                WML_SERVICE_URL = f"https://{l[1]}.ml.cloud.ibm.com"
            else:
                WML_SERVICE_URL = "https://us-south.ml.cloud.ibm.com"
            print(hostname)
            wml_credentials = {"apikey": token, "url": WML_SERVICE_URL}
        else:
            print("Failed to retrieve WML client")

        client = APIClient(wml_credentials)
        print("WML client initialized")

        return client
    

    # Logger class for updating Elasticsearch log index
    class rag_logger:
        
        def __init__(self, parameters):
            if not 'log_elastic_search_index_name' in parameters or parameters['log_elastic_search_index_name'] == '':
                self.logger_status = f"logging is deactivated"
                self.log_es_client = None
            else:
                self.log_elastic_search_index_name = parameters['log_elastic_search_index_name']
                try:
                    self.log_es_connection = client.connections.get_details(parameters['log_connection_id'])['entity']['properties']
                except:
                    self.log_es_connection = {key[19:]: parameters[key] for key in parameters.keys() if key.startswith('log_elastic_search_')}
                try:
                    self.log_es_client = rag_helper_functions.create_elastic_client(self.log_es_connection)
                    idx_create_status = self.log_es_client.options(ignore_status=400).indices.create(index=self.log_elastic_search_index_name)
                    self.logger_status = f"logger: connection established, log index {self.log_elastic_search_index_name} {'created' if not 'status' in idx_create_status else 'already exists' if idx_create_status['status']==400 else 'DEFECTIVE'}"
                except Exception as e:
                    self.logger_status = f"logger: ERROR: {str(e)}"
                    self.log_es_client = None
                    
                # Personally identifiable information (PII) detection
                self.pii_detection_model = None
                pii_status = 'inactive'
                self.pii_model_syntax = None
                self.pii_model_bilstm = None
                self.pii_model_rbr    = None
                if watson_nlp_available:
                    try:
                        self.pii_model_syntax = watson_nlp.load('syntax_izumo_en_stock')
                        # Bidirectional Long Short-Term Memory (BiLSTM) model for names detection
                        self.pii_model_bilstm = watson_nlp.load('entity-mentions_bilstm_en_pii')
                        # Rule-Based Reasoner (RBR) for credit / social numbers and email detection
                        self.pii_model_rbr = watson_nlp.load('entity-mentions_rbr_multi_pii')
                        pii_status = 'active'
                    except:
                        pii_status = 'model load failed'
                self.logger_status = self.logger_status + f", PII suppression status: {pii_status}"
                
                
        def status(self):
            return self.logger_status if self.logger_status else 'unknown'
                
        def remove_pii(self, text):
            # remove PII from given text
            if self.pii_model_bilstm and self.pii_model_rbr:
                for model_run in [lambda text: self.pii_model_bilstm.run(self.pii_model_syntax.run(text)), lambda text: self.pii_model_rbr.run(text, language_code='en')]:
                    pii_detection = model_run(text)
                    begin = 0
                    parts = []
                    for _pii in pii_detection.mentions:
                        parts.append(text[begin:_pii.span.begin])
                        begin = _pii.span.end
                    parts.append(text[begin:len(text)])
                    text = 'XXX'.join(parts)
            return text
            
        def create_log_record(self, question, content, times, feedback):
            log_id = ''
            if self.log_es_client:
                # compile LOG record (RAG response + question + time meassurements)
                log_record = { **content, 'log_timestamp': datetime.now().isoformat() }
                if question:
                    log_record['question'] = self.remove_pii(question)  # remove PII from user's question ("Hi I am Joe ...")
                if times:
                    log_record['elapsed_times'] = { times[i][0]: "{:.3f}".format(times[i][1] - times[i-1][1]) for i in range(1, len(times)) }
                if feedback:
                    log_record['feedback'] = feedback
                if content and 'response' in content:
                    log_record['response'] = self.remove_pii(log_record['response'])  # remove PII if genAI refers to user's question ("Hi Joe, ...")
                res = None
                res = self.log_es_client.index(index=self.log_elastic_search_index_name, document=log_record)
                if res != None:
                    log_id = res['_id']
                    status = 'ok'
                else:
                    status = 'log cannot be written'
                
            else:
                status = self.logger_status
                
            return status, log_id
        
        
        def update_log_record(self, feedback):
            
            if self.log_es_client:
                if not 'log_id' in feedback:
                    status = "log id not provided"
                else:
                    log_id = feedback['log_id']
                    feedback.pop("log_id")
                    if len(feedback.keys()) < 1:
                        # no feedback values
                        status = "feedback not provided"
                    else:
                        try:
                            # update document
                            idx_upd_status = self.log_es_client.options(ignore_status=404).update(index=self.log_elastic_search_index_name, id=log_id, doc={'feedback': feedback})
                            status = "ok" if not 'status' in idx_upd_status else ( f"log id {str(log_id)} not found" if idx_upd_status['status']==404 else f"update failed, status {str(idx_upd_status['status'])}" )
                        except Exception as e:
                            status = f"update of index {self.log_elastic_search_index_name} failed"
            else:
                status = self.logger_status
                
            return status
        
    
    def validate_answer_against_sources(response_answer, source_documents,similarity_threshold = 0.5):
        model = SentenceTransformer('all-MiniLM-L6-v2')
        source_texts = [doc.page_content for doc in source_documents]
        answer_embedding = model.encode(response_answer, convert_to_tensor=True)
        source_embeddings = model.encode(source_texts, convert_to_tensor=True)
        cosine_scores = util.pytorch_cos_sim(answer_embedding, source_embeddings)
        print(cosine_scores[0])
        if any(score.item() > similarity_threshold for score in cosine_scores[0]):
            matching_scores = [score.item() for score in cosine_scores[0] if score.item() > similarity_threshold]
            confidence_score = max(matching_scores) 
            return {'isHallucination': False,'confidence_score': round(float(confidence_score),2)}
        else:
            confidence_score = sum(cosine_scores[0]) / len(source_documents)
            print(confidence_score)
            return {'isHallucination': True,'confidence_score': round(float(confidence_score),2)}

    def is_hallucination(response_answer,source_documents, threshold_overlap_max = 0.3, threshold_overlap_score_concat = 0.4):
        stop_words = {
            'a', 'about', 'after', 'all', 'also', 'am', 'an', 'and', 'another', 'any', 'are', 'as', 'at',
            'be', 'because', 'been', 'before', 'being', 'between', 'both', 'but', 'by',
            'came', 'can', 'come', 'could', 'did', 'do', 'each', 'for', 'from','get', 'got',
            'has', 'had', 'he', 'have', 'her', 'here', 'him', 'himself', 'his', 'how',
            'i', 'if', 'in', 'into', 'is', 'it', 'like',
            'make', 'many', 'me', 'might', 'more', 'most', 'much', 'must', 'my', 'never', 'now',
            'of', 'on', 'only', 'or', 'other', 'our', 'out', 'over',
            'said', 'same', 'should', 'since', 'some', 'still', 'such',
            'take', 'than', 'that', 'the', 'their', 'them', 'then', 'there', 'these', 'they', 'this',
            'those', 'through', 'to', 'too', 'under', 'up', 'very', 'was', 'way', 'we', 'well', 'were', 
            'what', 'where', 'which', 'while', 'who', 'with', 'would', 'you', 'your'
        }

        regex = re.compile(r"\b\w+(?:['-_]\w+)?\b")

        def calculate_textual_overlap(text1, text2):
            # tokenize
            text1_tokens = regex.findall(text1.lower())
            text2_tokens = regex.findall(text2.lower())

            # remove stop words
            text1_tokens = [t for t in text1_tokens if t not in stop_words]
            text2_tokens = [t for t in text2_tokens if t not in stop_words]

            # compute overlap
            if len(text1_tokens) > 0:
                text1_tokens = set(text1_tokens)
                text2_tokens = set(text2_tokens)
                return len(text1_tokens.intersection(text2_tokens)) / len(text1_tokens)
            else:
                return 0

        def overlap_score_concat(generated_text, passages):
            passages_text = ' '.join(passages)
            return calculate_textual_overlap(generated_text, passages_text)


        def overlap_score_max(generated_text, passages):
            return max([calculate_textual_overlap(generated_text, passage_text) for passage_text in passages])

        # Get the LLM response along with source document text
        llm_response_text = response_answer
        source_docs_passages = [doc.page_content for doc in source_documents]

        result_overlap_score_max = 0.0
        result_overlap_score_concat = 0.0

        if len(source_docs_passages):

            # Call overlap score calculations
            result_overlap_score_max = overlap_score_max(llm_response_text, source_docs_passages)
            result_overlap_score_concat = overlap_score_concat(llm_response_text, source_docs_passages)

            print('result_overlap_score_max: '+ str(result_overlap_score_max))
            print('result_overlap_score_concat: ' + str(result_overlap_score_concat))


            # Define thresholds (these should be put in a parameter set or template)
            # Need to experiment with what values to use

            response = {'isHallucination': False,
                        'text': "",
                        'maxOverlapScore': round(result_overlap_score_max,2),
                        'concatOverlapScore': round(result_overlap_score_concat,2)}

            # Decide if hallucination or not
            if result_overlap_score_max > threshold_overlap_max and result_overlap_score_concat > threshold_overlap_score_concat:
                
                response['isHallucination'] = False
            else:
                print("Hallucination")
                response['isHallucination'] = True  
                response['text'] = "Sorry, I can not generate a valid answer to your question."
        else:
                response = {'isHallucination': True,
                'text': "Sorry, I can not find an answer to your question in the available documents.",
                'maxOverlapScore': round(result_overlap_score_max,2),
                'concatOverlapScore': round(result_overlap_score_concat,2)}

        return response      
    
    try:
        if environment == "cloud":
            client = get_WML_client(environment, ibm_api_key, hostname)
        elif environment == "on-prem":
            client = get_WML_client(environment, token, hostname)
        client.set.default_space(params['space_id'])
    except Exception as e:
        
        print(f"Error initializing WML client: {str(e)}")
          
        raise

        
    try:
        helper_function_name = 'rag_helper_functions.py'
        helper_function_path = client.data_assets.download(space_asset_dict[helper_function_name],helper_function_name)
        script_name = os.path.basename(helper_function_path).replace('.py', '')
        rag_helper_functions = __import__(script_name)
        space_parameter_dict = {key: value for key, value in space_asset_dict.items() if "parameter" in key.lower()}


        parameters_list = [] 
        for i in space_parameter_dict.values():
            parameters_list= parameters_list+client.parameter_sets.get_details(i)['entity']['parameter_set']['parameters']
        parameters = {param['name']: param['value'] for param in parameters_list}
        


        hallucination_threshold_max_text_overlap = parameters['hallucination_threshold_max_text_overlap']
        hallucination_threshold_concatenated_text_overlap = parameters['hallucination_threshold_concatenated_text_overlap']
  
        
    except Exception as e:
  
        print(f"Error initializing the parameters: {str(e)}")
        raise
    
    try:
        es_connection = client.connections.get_details(params['connection_id'])['entity']['properties']
        es_client=rag_helper_functions.create_elastic_client(es_connection)
        
    except Exception as e:
            
        print(f"Error creating elastic search connection: {str(e)}")
        

    
        # Retrieve indices name
    
    es_index = parameters["elastic_search_index_name"]
    model_id = parameters['elastic_search_model_id']


    
    vector_store = VectorStore(
    client=client,
    connection_id=params['connection_id'],
    data_source_type='elasticsearch',
    params={ 
         "index_name":parameters["elastic_search_index_name"],
         "model_id": parameters['elastic_search_model_id'],
         "vector_query_field":parameters['vector_query_field']
     }
    )

    try:


    # Gen AI via WatsonX.AI
        if parameters['watsonx_ai_api_key']:
            print("Running the prompt on watsonx.ai cloud ..")
            
            model_credentials = Credentials(
                url=parameters['watsonx_ai_url'],
                api_key=parameters['watsonx_ai_api_key']
            )
            model_credentials.token = None

        else:
            print("Running the prompt on watsonx.ai on-prem ..")
            model_credentials =  Credentials(
                            token=token,
                            url=hostname,
                            instance_id='openshift'
            )

            
        if environment == "cloud":
            prompt_credentials = model_credentials
        elif environment == "on-prem":
            prompt_credentials = client.wml_credentials
            
        print("Reading Prompt")  
        prompt_mgr = PromptTemplateManager(
            credentials=prompt_credentials,
            space_id=params['space_id'])
        
        df_prompt = prompt_mgr.list()
        prompt_template_id=df_prompt.loc[df_prompt['NAME'] == parameters['llm_prompt_template_file'], 'ID'].values[0]
        prompt_model = prompt_mgr.load_prompt(prompt_template_id) 

        if 'stop_sequences' in prompt_model.model_params:
            prompt_model.model_params['stop_sequences']=['<|endoftext|>']

        model = Model(
            model_id=prompt_model.model_id,
            params=prompt_model.model_params,
            credentials=model_credentials,
            project_id=parameters['watsonx_ai_project_id']
        )
        

        print("model initialized")
        QA_CHAIN_PROMPT = prompt_mgr.load_prompt(prompt_template_id, PromptTemplateFormats.LANGCHAIN)
    
        #retriever = vector_store.as_langchain_retreiver(search_type="similarity_score_threshold",search_kwargs={"k":parameters['rag_es_top_n_results'],"score_threshold":float(parameters['rag_es_min_score'])})
        
        def retrieve_with_scores(query, filter=None):
            search_kwargs = {
                "k": parameters['rag_es_top_n_results'],
                "score_threshold": float(parameters['rag_es_min_score']),
                "include_scores": True,
                "verbose": True
            }
            if filter is not None:
                search_kwargs["filter"] = filter
 

            results = vector_store.search(query, **search_kwargs)

            return [{"document": doc, "score": score} for doc, score in results if score > float(parameters['rag_es_min_score'])]

        def format_docs(docs_with_scores):
            return "".join([f"[Document]\n{d['document'].page_content}[End]\n\n" for d in docs_with_scores])

        print("retriever created")
        llm_chain = (
            RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
            | QA_CHAIN_PROMPT
            | model.to_langchain()
            | StrOutputParser()
        )
        print("llm chain created")

        def retrieve_with_scores_wrapper(inputs):
            query = inputs["query"]
            filter = inputs.get("filter")
            return retrieve_with_scores(query, filter)

        rag_chain_with_source = RunnableParallel({
            "context": retrieve_with_scores_wrapper,
            "question": RunnablePassthrough(),
        }).assign(answer=llm_chain)
        print("rag chain created")

  
        
    except Exception as e:
            
        print(f"Error creating LLM Chain: {str(e)}")


    logger = rag_logger({**parameters, **params})
    print(f"{logger.status()}")
    
    def score(payload):
        
        # payload scheme:
        #
        #   Run RAG pipeline:
        #     { fields: [<anything other than "log_id", "_log", "_content">], values [[<question>]] }
        #
        #   Run RAG pipeline without logging:
        #     { fields: [<anything other than "log_id", "_log", "_content">, "_log"], values [[<question>, False]] }
        #
        #   Update attribute "feedback" in log: 
        #     { fields: ["log_id", "field1", ..., "fieldN"], values: [[<log_id>, value1, ..., valueN]] }
        #
        #   Run RAG pipeline with pre-filled "feedback" in log:
        #     { fields: ["question", "field1", ..., "fieldN"], values: [[<question>, value1, ..., valueN]] }
        #
        #   Create log record:
        #     { fields: ["_content"], values: [[<content>]] }
        
        logging = True
        question = None
        query_filter = None
        status = f"invalid parameters"
        fields = {payload['input_data'][0]['fields'][i]: payload['input_data'][0]['values'][0][i] for i in range(min(len(payload['input_data'][0]['fields']), len(payload['input_data'][0]['values'][0])))}
        
        
        if '_log' in fields:
            logging = fields.pop('_log')
            
        if 'question' in fields:
            question = fields.pop('question')
            
        if '_content' in fields:
            (status, log_id) = logger.create_log_record(None, fields['_content'], None, None) if logger else (f"log not available", '')
            return {"predictions": [ { "fields": ["status", "log_id"], "values": [[status, log_id]] } ]}
                                 
        if 'log_id' in fields:
            status = logger.update_log_record(fields) if logger else f"log not available"
            
        if 'query_filter' in fields:
            query_filter = fields.pop('query_filter')
            
        else:
        
            if not question and len(fields.keys()) == 1:          
                question = payload['input_data'][0]['values'][0][0]
                fields = {}

        if not question:
            return {"predictions": [ { "fields": ["status"], "values": [[status]] } ]}


        # initialize time measurement list
        times = [('start', time.perf_counter())]
        
        try:
            
            if query_filter:
                
                inputs = {
                    "query": question,
                    "filter": [{"match": query_filter}]
                }
            else:
                inputs = {"query": question}
            

            llm_response = rag_chain_with_source.invoke(inputs)

            
            
            times.append(('llm_chain', time.perf_counter()))
            scoring_response_txt={}
            scoring_response_txt['response'] = llm_response['answer']
            

            hallucination_response_text ="Sorry, I cannot find an answer to your question in the available documents."
            
            
            if llm_response['context']:
                llm_response_documents=[i['document'] for i in llm_response['context']]
                
                if parameters['default_hallucination_technique'] == "sentence_transformer":
                    hallucination_response = validate_answer_against_sources(llm_response['answer'], llm_response_documents, parameters["similarity_threshold_for_sentence_transformer_hallucination"])
                    hallucination_dict={"Technique":"sentence_transformer","confidence_score":hallucination_response['confidence_score']}
                elif parameters['default_hallucination_technique'] == "word_overlap":
                    hallucination_response=is_hallucination(llm_response['answer'], llm_response_documents,hallucination_threshold_max_text_overlap, hallucination_threshold_concatenated_text_overlap)
                    hallucination_dict={"Technique":"Word Overlap","text_max_overlap_score":hallucination_response['maxOverlapScore'],"concatenated_text_overlap_score":hallucination_response['concatOverlapScore']}

                else:
                    hallucination_dict={"Technique":"None"}
                    hallucination_response={"isHallucination":False}
                if hallucination_response['isHallucination']:
                    scoring_response_txt['response']=hallucination_response_text
                    
                    
                
                scoring_response_txt['source_documents'] = [{**response['document'].dict(), 'score': response['score']} for response in llm_response['context']]#[doc for doc in llm_response['context']]
                
                scoring_response_txt['Hallucination Detection'] =hallucination_dict
                    
                
            else:
                scoring_response_txt['response']=hallucination_response_text
                scoring_response_txt['Hallucination Detection'] ={"Technique":parameters['default_hallucination_technique']}

                    
            times.append(('hallucination_detection', time.perf_counter()))
     


        except Exception as e:
            
            print(f"Error while getting LLM response: {str(e)}")
            raise
      
        # post log record and return log id to caller
        if logging:
            scoring_response_txt['log_status'], scoring_response_txt['log_id'] = logger.create_log_record(question, scoring_response_txt, times, fields)

        scoring_response = {
            "predictions": [
                {
                    "fields": [
                        "llm_prediction"
                    ],
                    "values": [[scoring_response_txt]]
                }]}
        
        return scoring_response
    
    return score

Uncomment the below cells to test the scoring pipeline function

In [ ]:
#rag_scoring_pipeline_function()({"input_data": [{"fields": ["question"],"values": [["How can I review model transactions in Watson Studio?"]]}]})

We can filter the source documents title with a value `SQL optimization (SPSS Modeler)` and LLM will generate the response from only those source documents using the below cell.

In [ ]:
#rag_scoring_pipeline_function()({"input_data": [{"fields": ["question", "query_filter"],"values": [["how to perform sql optimization?",{"metadata.title.keyword": {"query": "SQL optimization (SPSS Modeler)"}}]]}]})

<a id="DeployScoringFunction"></a>
### Deployment of function to Space

Initialize the function details and the metadata.

In [ ]:

pyfunc_swspec_id = client.software_specifications.get_id_by_name("runtime-23.1-py3.10")

meta_data = {
    client.repository.FunctionMetaNames.NAME: 'rag_function_watsonx_ai',
    client.repository.FunctionMetaNames.DESCRIPTION: 'Watsonx_Function_for_RAG_with_ElasticSearch',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: pyfunc_swspec_id
}

watsonx_function_details = client.repository.store_function(meta_props=meta_data, function=rag_scoring_pipeline_function)


Create the function deployment in the space

In [ ]:
watsonx_function_uid = client.repository.get_function_id(watsonx_function_details)

meta_props = {
   client.deployments.ConfigurationMetaNames.NAME: "rag-function-watsonx-ai-deployment",
   client.deployments.ConfigurationMetaNames.DESCRIPTION: "Wrapper function to perform RAG",
   client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S'},  
   client.deployments.ConfigurationMetaNames.SERVING_NAME: "rag_with_wx_es" + str(uuid.uuid1()).split('-')[0]
}

watsonx_deployment_details = client.deployments.create(watsonx_function_uid, meta_props=meta_props)

<a id="scoring"></a>
### Test the scoring function

Test the deployed scoring function by passing in a question.

In [ ]:
watsonx_deployment_id = client.deployments.get_id(watsonx_deployment_details)

payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "fields": ["question"],
        "values": [["how to perform sql optimization on watsonx.ai?"]]
       
    }]
}
wx_result = client.deployments.score(watsonx_deployment_id, payload)
wx_result

The response above can be filtered by using a `query_filter` for the question. The `query_filter` ensures that the responses come from specific documents in the Elastic Search index. The format for `query_filter` is as follows:
```
{"<field_to_filter>": {"query": "<value of the field>"}}
```

Below cell demonstrates a sample response using a `query_filter`.

In [ ]:
payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "fields": ["question", "query_filter"],
        "values": [["how to perform sql optimization?",{"metadata.title.keyword": {"query": "SQL optimization (SPSS Modeler)"}}]]
        
       
    }]
}
wx_result = client.deployments.score(watsonx_deployment_id, payload)
wx_result

<a id="feedback-logging"></a>
### Feedback Logging

Updating `feedback` in log record. <br>
In case the answer returned by the LLM is relevant to your question provide the values below as `postive` along with a comment.
Otherwise, provide the values below as `negative`. 

This feedback will be saved if a elastic search index is provided for feedback logging in the parameter set. You can view the indexed feedback logs via the elastic search kibana user interface under the **Index Management** section. Find the document for the log record that corresponds to the conversation above. It contains, among others, the question and response, documents found and the user's feedback.

In [ ]:
try:
    log_id = wx_result['predictions'][0]['values'][0][0]['log_id']
except NameError: 
    log_id = ''
    
payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "fields": ["log_id", "value", "comment"],
        "values": [[log_id, "positive", "Nice log record."]]
       
    }]
}
wx_log_update_result = client.deployments.score(watsonx_deployment_id, payload)
wx_log_update_result

<a id="updateParameters"></a>
### Update parameter set in the project & deployment space
Update the advanced parameter set in both project & sapce with the deployment id of the function.

In [ ]:
paramset_name = "RAG_advanced_parameter_set"
parameter_to_be_updated = {"name":"wml_rag_deployment_id","value":watsonx_deployment_id}
rag_helper_functions.update_parameter_set(client,paramset_name,parameter_to_be_updated)

client.set.default_project(project_id=project_id)
if rag_helper_functions.update_parameter_set(client,paramset_name,parameter_to_be_updated) == True:
    print("Parameter set in the project and deployment space has been updated with the deployment id of the function above.")
else:
    print("Parameter set update failed.")

 Proceed to **Q&A with RAG** notebook to test out the deployed function by asking some questions.

**Sample Materials, provided under license.</a> <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2024. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>**